In [28]:
import  seaborn as sns

In [29]:
df=sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [30]:
df['day'].value_counts()

day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64

In [31]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [32]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [33]:
df['day'].unique()

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

In [34]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [35]:
df.time

0      Dinner
1      Dinner
2      Dinner
3      Dinner
4      Dinner
        ...  
239    Dinner
240    Dinner
241    Dinner
242    Dinner
243    Dinner
Name: time, Length: 244, dtype: category
Categories (2, object): ['Lunch', 'Dinner']

In [36]:
from sklearn.preprocessing import LabelEncoder

In [37]:
encoder=LabelEncoder()
df['time']=encoder.fit_transform(df['time'])

In [38]:
df.time.unique()

array([0, 1])

In [39]:
#independent and dependent feature
x=df.drop(labels=['time'],axis=1)
y=df.time

In [40]:
x.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [41]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int64

In [42]:
from sklearn.model_selection import train_test_split

In [43]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)

In [44]:
from sklearn.pipeline import Pipeline

In [45]:
from sklearn.impute import SimpleImputer #for handelling missing values

In [46]:
from sklearn.preprocessing import StandardScaler #for feature engineering

In [47]:
from sklearn.preprocessing import OneHotEncoder

In [48]:
from sklearn.compose import ColumnTransformer

In [53]:
categorical_cols=['sex',"smoker",'day']
numerical_cols=['total_bill','tip','size']

In [57]:
#feature engineering automation
#numerical pipelines
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),#missing value handeled
        ('scaler',StandardScaler()) #feature scaling
    ]

)

#categorical pipeline
cat_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')), #applied mode for missing value
        ('onehotencoder',OneHotEncoder()) #catogiracl features to numerical
    ]

)

In [58]:
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
])

In [59]:
x_train=preprocessor.fit_transform(x_train)
x_test=preprocessor.transform(x_test)

In [78]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [79]:

#model training automation
models={
    'Random Forest': RandomForestClassifier(),
    'Decision Tree ':DecisionTreeClassifier(),
    'Logistic regression': LogisticRegression()
}

In [80]:
from sklearn.metrics import accuracy_score

In [81]:
def evaluate_model(x_train,x_test,y_train,y_test,models):
  report={}
  for i in range (len(models)):
    model=list(models.values())[i]
    #train model
    model.fit(x_train,y_train)

    #predict testing data
    y_test_pred=model.predict(x_test)

    #get accuracy score
    test_model_score=accuracy_score(y_test,y_test_pred)

    report[list(models.keys())[i]]=test_model_score

  return report

In [82]:
results = evaluate_model(x_train, x_test, y_train, y_test, models)
print(results)

{'Random Forest': 0.9591836734693877, 'Decision Tree ': 0.9387755102040817, 'Logistic regression': 1.0}


In [83]:
def evaluate_model(x_train, x_test, y_train, y_test, models):
    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        # Train model
        model.fit(x_train, y_train)

        # Predict testing data
        y_test_pred = model.predict(x_test)

        # Get accuracy score
        test_model_score = accuracy_score(y_test, y_test_pred)

        report[list(models.keys())[i]] = test_model_score

    return report

# Evaluate the model
results = evaluate_model(x_train, x_test, y_train, y_test, models)
print(results)


{'Random Forest': 0.9591836734693877, 'Decision Tree ': 0.9387755102040817, 'Logistic regression': 1.0}


In [84]:
# # to get best model score from dictionary

# best_model_score=max(sorted(model_report.values()))

NameError: name 'model_report' is not defined

In [86]:
classifier=RandomForestClassifier()

In [87]:
#hyperparameter tuning
params={'max_depth':[3,5,10,None],
        'n_estimators':[100,200,300],
        'criterion':['gini','entropy']
        }

In [88]:
from sklearn.model_selection import RandomizedSearchCV

In [93]:
cv=RandomizedSearchCV(classifier,param_distributions=params,scoring='accuracy',cv=5,verbose=3)

In [94]:
cv.fit(x_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.974 total time=   0.5s
[CV 2/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.949 total time=   0.4s
[CV 3/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.974 total time=   0.5s
[CV 4/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.923 total time=   0.5s
[CV 5/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.923 total time=   0.4s
[CV 1/5] END criterion=entropy, max_depth=None, n_estimators=300;, score=0.974 total time=   0.6s
[CV 2/5] END criterion=entropy, max_depth=None, n_estimators=300;, score=0.923 total time=   0.8s
[CV 3/5] END criterion=entropy, max_depth=None, n_estimators=300;, score=1.000 total time=   0.7s
[CV 4/5] END criterion=entropy, max_depth=None, n_estimators=300;, score=0.949 total time=   0.8s
[CV 5/5] END criterion=entropy, max_depth=None, n_estimators=300;, score

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [95]:
cv.best_params_

{'n_estimators': 300, 'max_depth': None, 'criterion': 'entropy'}